# Steered MD in BioSimSpace

Allosteric inhibition can be a useful alternative to conventional protein target-ing when the nature of the active site makes it difficult to design binders. This requires  assessment  of  whether  an  allosteric  binder  actually  has  an  effect  onprotein function, such as whether its presence shifts protein conformational ensemble to favour the inactive state. This can be modelled as a Markov chain by Markov State Models(MSMs).  Since the system is treated as memoryless, model building only requires local equilibrium. Therefore, it can make use of shorter MD simulations, allowing them to be run in parallel.

In order to have a more complete view of the protein ensemble, enhanced sampling methods are used, among them steered MD (sMD) (1). It introduces a bias potential that is added to the Hamiltonian, thus biasing the simulation towards a specified value of a chosen collective variable. Once the system has reached a certain conformation, those coordinates (2) can be used as starting points for equilibrium MD simulations (4) that can subsequently be used as data for constructing an MSM (4). An example summary of this is shown below:
<img src="figures/ensemble-md-protocol.png" width=300>

PLUMED is a library that, among other things, has enhanced sampling algorithms. It works with multiple MD engines, including GROMACS and AMBER. PLUMED uses a [moving restraint](https://www.plumed.org/doc-v2.5/user-doc/html/_m_o_v_i_n_g_r_e_s_t_r_a_i_n_t.html) that is calculated as follows:

$V(\vec{s},t) = \frac{1}{2} \kappa(t) ( \vec{s} - \vec{s}_0(t) )^2$     (Eq. 1)

where $\vec{s}_0$ and $\kappa$ are time dependent and specified in the PLUMED input. $\vec{s}_0$ is the target CV value and $\kappa$ is the force constant in kJ mol$^{-1}$. The values of both of them are set at specific steps, and linearly interpolated in between.

This tutorial focuses on running the prerequisite simulations using BSS.The example system used is protein tyrosine phosphatase 1B(PTP1B), which exists in two dominant conformations: WPD loop open and WPD loop closed:
<img src="figures/open-close.png" width=250>

## set up sMD

Running steered MD in BioSimSpace is very similar to regular simulations already covered. It only requires some more preparation for interfacing with PLUMED, the software that takes care of biasing the Hamiltonian.

#### Setting up the system

We start by importing the required libraries:

In [1]:
import BioSimSpace as BSS
import os
from shutil import copyfile

Load a system with BioSimSpace. This particular system is of PTP1B with the WPD loop open (from PDB entry 2HNP) and has been minimised and equilibrated.

In [2]:
system = BSS.IO.readMolecules(['data/system.prm7', 'data/system.rst7'])

#### Creating the CV

Steered MD uses a specific CV, which in this case is RMSD of the WPD loop (residues 178-184). To calculate RMSD, we specify a reference structure first:

In [3]:
reference = BSS.IO.readMolecules('data/reference.pdb').getMolecule(0)

In [4]:
rmsd_indices = []
for residue in reference.getResidues():
    if 178<=residue.index()<=184:
        for atom in residue.getAtoms():
            if atom.element()!='Hydrogen (H, 1)':
                rmsd_indices.append(atom.index())

In [5]:
rmsd_cv = BSS.Metadynamics.CollectiveVariable.RMSD(system, reference, 0, rmsd_indices)

#### Setting up a steered MD protocol

To create the protocol, we need to set up the restraints and a steering schedule. The steering schedule will set the steps that will have the values of $\kappa(t)$ and $\vec{s}_0(t)$ set in the restraints, as seen in equation 1. The steps involve the starting step, applying the steering force over a short period of time, the steering itself (the bulk of the simulation) and a short relaxation period. 

In [6]:
start = 0* BSS.Units.Time.nanosecond
apply_force = 4 * BSS.Units.Time.picosecond
steer = 150 * BSS.Units.Time.nanosecond
relax = 152 * BSS.Units.Time.nanosecond

The restraints specify the expected CV value and the force constant ($\kappa(t)$ and $\vec{s}_0(t)$) at each step created above.

In [7]:
nm = BSS.Units.Length.nanometer
restraint_1 = BSS.Metadynamics.Restraint(rmsd_cv.getInitialValue(), 0)
restraint_2 = BSS.Metadynamics.Restraint(rmsd_cv.getInitialValue(), 3500)
restraint_3 = BSS.Metadynamics.Restraint(0*nm, 3500)
restraint_4 = BSS.Metadynamics.Restraint(0*nm, 0)

In [8]:
protocol = BSS.Protocol.Steering(rmsd_cv, [start, apply_force, steer, relax], [restraint_1, restraint_2, restraint_3, restraint_4], runtime=152*BSS.Units.Time.nanosecond)

#### A quick look at GROMACS

We have previously created a protocol for sMD, so all that is needed is to plug it into a GROMACS process

In [10]:
process = BSS.Process.Gromacs(system, protocol)

We can have a look at the command arguments that will be used to run this simulation:

In [11]:
process.getArgs()

OrderedDict([('mdrun', True),
             ('-v', True),
             ('-deffnm', 'gromacs'),
             ('-plumed', 'plumed.dat')])

The argument `-plumed plumed.dat` tells GROMACS to use PLUMED, looking at the `plumed.dat` file for instructions. This process can be run like any other process you have seen before. All the required files have been created in the `process.workDir()` by BioSimSpace.

#### Steered MD in AMBER

Just as with GROMACS, we simply need to create a process in AMBER:

In [9]:
process = BSS.Process.Amber(system, protocol)

Check the configuration of the process:

In [11]:
process.getConfig()

['Production.',
 ' &cntrl',
 '  ig=-1,',
 '  ntx=1,',
 '  ntxo=1,',
 '  ntpr=1000,',
 '  ntwr=1000,',
 '  ntwx=1000,',
 '  irest=0,',
 '  dt=0.002,',
 '  nstlim=76000000,',
 '  ntc=2,',
 '  ntf=2,',
 '  ntt=3,',
 '  gamma_ln=2,',
 '  cut=8.0,',
 '  tempi=300.00,',
 '  temp0=300.00,',
 '  ntp=1,',
 '  pres0=1.01325,',
 '  plumed=1,',
 "  plumedfile='plumed.dat',",
 ' /']

The lines `plumed=1` and `plumedfile="plumed.in"` are what specify that PLUMED will be used. The process can now be started to run steered MD.

**Note**: when BioSimSpace writes the new topology file for an AMBER process, some unidentified small change causes PTP1B to stat unfolding on the timescale of a few ns. This is circumvented by simply copying over the original topology.

In [16]:
copyfile('data/system.prm7', f'{process.workDir()}/amber.prm7')

'/tmp/tmps7klsay_/amber.prm7'